## 3. Implementación de Modelos:

### Cargamos liberías

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# Optimización y entrenamiento
from sklearn.model_selection import train_test_split, GridSearchCV

# Modelos
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Preprocesamiento
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Métricas
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
# Importamos el df del eda
df = pd.read_pickle('df_data.pkl')
df.head(5)

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,year,month
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,2023,11
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,2023,2
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,2023,1
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,2023,5
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,2023,5


### Organización del conjunto de datos y entrenamiento 

In [13]:
# Definimos la matriz de caracteristicas (X) y nuestra variable objetivo (y)
X = df.drop(columns=['Quantity', 'Transaction ID', 'Customer ID','year','month'])
y = df['Quantity']

# Dividimos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definimos las columnas categoricas y númericas
numeric_features = ['Age', 'Price per Unit','Total Amount']
categorical_features = ['Gender', 'Product Category']

# Preprocesador
# Aplicamos OneHoteEncoder a los columnas categoricas
# Aplicamos  StandardScaler a las columnas númericas
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(), categorical_features)
])

# Creamos pipelines para los modelos
tree_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

forest_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])


# Definimos hiperparámetros para GridSearchCV
tree_param_grid = {
    'model__max_depth': [3, 5, 10],
    'model__min_samples_split': [2, 5, 10]
}

forest_param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [3, 5, 10],
    'model__min_samples_split': [2, 5, 10]
}

# Optimizamos con GridSearchCV
tree_grid = GridSearchCV(tree_pipeline, tree_param_grid, cv=5, scoring='neg_mean_squared_error')
forest_grid = GridSearchCV(forest_pipeline, forest_param_grid, cv=5, scoring='neg_mean_squared_error')

# Entrenamos los modelos optimizados
tree_grid.fit(X_train, y_train)
forest_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'Price '
                                                                          'per '
                                                                          'Unit',
                                                                          'Total '
                                                                          'Amount']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['Gender',
                                                                          'Product '
                                                                          'Category'])])),
                                       ('model',
                                        RandomForestRegressor(random_state=42))]),
             param_grid={'model__max_depth': [3, 5, 10],
                         'model__min_samples_split': [2, 5, 10],
                         'model__n_estimators': [100, 200]},
             scoring='neg_mean_squared_error')

**Nota:** No se tuvo en cuenta las columnas 'Transaction ID' y 'Customer ID' porque considero que no van a aportar información relevante al modelo, en cuanto a las fechas como vimos en el EDA las ventas son muy equilibradas a lo largo de los meses, mantener la fecha podría ser valioso si tuvieramos más datos que comprendieran más años y así poder comprender si existe una estacionalidad en las ventas, pero en nuestro caso como solo son datos de un año no mantendremos la columna fecha.

### Predicciones y evaluación de modelos

In [15]:
# Hacemos las predicciones
tree_preds = tree_grid.predict(X_test)
forest_preds = forest_grid.predict(X_test)

# Evaluamos los modelos
tree_mse = mean_squared_error(y_test, tree_preds)
forest_mse = mean_squared_error(y_test, forest_preds)

tree_rmse = np.sqrt(tree_mse)
forest_rmse = np.sqrt(forest_mse)

tree_r2 = r2_score(y_test, tree_preds)
forest_r2 = r2_score(y_test, forest_preds)

# Resultados
print("Decision Tree - MSE:", tree_mse, "RMSE:", tree_rmse, "R^2:", tree_r2)
print("Random Forest - MSE:", forest_mse, "RMSE:", forest_rmse, "R^2:", forest_r2)

# Mejor hiperparámetro para cada modelo
print("Mejores hiperparámetros Decision Tree:", tree_grid.best_params_)
print("Mejores hiperparámetros Random Forest:", forest_grid.best_params_)

Decision Tree - MSE: 0.0 RMSE: 0.0 R^2: 1.0
Random Forest - MSE: 5.833333333333085e-07 RMSE: 0.000763762615825957 R^2: 0.9999995527004115
Mejores hiperparámetros Decision Tree: {'model__max_depth': 10, 'model__min_samples_split': 2}
Mejores hiperparámetros Random Forest: {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 200}


## Análisis y conclusiones 

**Decisión Tree:** Muestra un ajuste perfecto lo cual puede indicar un sobreajuste esto debido a que los arboles de decisión tienden a memorizar los datos de entranmiento y por lo tanto pueden tener un buen rendimiento en el entrenamiento pero no generalizar bien los datos nuevos. Además con el análisis que hicimos desde el EDA en cuanto a la distribución de los datos de las diferentes caracteristicas podemos intuir que el conjunto de prueba para el modelo no debe presentar mayor problema en cálculo.

**Random forest:** Tiene un R cuadrado casi perfecto y valores de MSE y RMSE muy bajos, si bien no es un ajuste perfecto genera mayor confianza por el hecho de que el Random Forest es más robusto al promediar múltiples árboles y es menos propenso al sobreajuse y por lo tanto lo considero un mejor modelo.